# Azure Machine Learning  

#  Automated ML vs HyperDrive


TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-138107
aml-quickstarts-138107
southcentralus
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# NOTE: update the key to match the dataset name
found = False
key = "heart-fail"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'      
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
# choose a name for experiment
experiment_name = 'aml-exp-heart'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',   
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             model_explainability=True,
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = ".",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on aml-compute with default configuration
Running on remote compute: aml-compute
Parent Run ID: AutoML_823e640e-dd5e-473e-8c0b-9f5efa0946d4

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
remote_run.wait_for_completion()

{'runId': 'AutoML_823e640e-dd5e-473e-8c0b-9f5efa0946d4',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-08T06:24:18.758068Z',
 'endTimeUtc': '2021-02-08T06:42:02.974959Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'aml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"aml-exp-heart","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-138107","workspace_name":"quick-starts-ws-138107","region":"southcentralus","compute_target":"aml-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"f

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
remote_run.get_metrics()

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetCrossValidationSplit',
  'ModelSelection'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Generating individually featurized CV splits.',
  'Beginning model selection.'],
 'matthews_correlation': 0.7292901226073087,
 'balanced_accuracy': 0.843095238095238,
 'recall_score_micro': 0.8763276836158193,
 'average_precision_score_macro': 0.9045373782280415,
 'precision_score_micro': 0.8763276836158193,
 'recall_score_weighted': 0.8763276836158193,
 'accuracy': 0.8763276836158193,
 'average_precision_score_weighted': 0.927638222849392,
 'log_loss': 0.3842299506449608,
 'weighted_accuracy': 0.8986147612977045,
 'f1_score_macro': 0.8490680142814344,
 'f1_score_weighted': 0.

In [11]:
best_run, fitted_model=remote_run.get_output()
print(best_run)
print(fitted_model)




Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: aml-exp-heart,
Id: AutoML_823e640e-dd5e-473e-8c0b-9f5efa0946d4_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.5789473684210527,
                                                                                                silent=True,
                             

In [12]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'aml-exp-heart\',\'compute_target\':\'aml-compute\',\'subscription_id\':\'f9d5a085-54dc-4215-9ba6-dad5d86e60a0\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_823e640e-dd5e-473e-8c0b-9f5efa0946d4_38","experiment_name":"aml-exp-heart","workspace_name":"quick-starts-ws-138107","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group_name":"aml-quickstarts-138107"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '38',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.ComputeTargetType

In [13]:
#TODO: Save the best model
 best_model=   best_run.register_model(model_name="automl_bestmodel",model_path="./outputs/model.pkl")

In [14]:
best_model

Model(workspace=Workspace.create(name='quick-starts-ws-138107', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-138107'), name=automl_bestmodel, id=automl_bestmodel:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
import sklearn
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment.from_conda_specification(name="env", file_path="./conda_env.yml")

for pip_package in ["scikit-learn"]:
    myenv.python.conda_dependencies.add_pip_package(pip_package)

In [16]:
myenv

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210104.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
        

In [17]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_lambda=0.5789473684210527,
                                                                                                silent=True,
                                                                                                subsample=0.24789473684210528,
                             

In [18]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_93f4e81a5be7bbb451de9626ed22aad33581b7ef37ceda1e639dbc66f3255d1a_d.txt',
 'azureml-logs/65_job_prep-tvmps_93f4e81a5be7bbb451de9626ed22aad33581b7ef37ceda1e639dbc66f3255d1a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_93f4e81a5be7bbb451de9626ed22aad33581b7ef37ceda1e639dbc66f3255d1a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/104_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [26]:
from azureml.core.model import Model
model = Model(ws, 'automl_bestmodel',version=1)
model

Model(workspace=Workspace.create(name='quick-starts-ws-138107', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-138107'), name=automl_bestmodel, id=automl_bestmodel:1, version=1, tags={}, properties={})

In [27]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
inference_config = InferenceConfig(entry_script='inference_score.py', environment=myenv)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                auth_enabled=True,
                                                tags = {'name':'heartfailure-aml-service'},
                                                description='Heart failure model',
                                                enable_app_insights=True)
service = Model.deploy(workspace=ws,name="automl-deploy",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [28]:
print(service.scoring_uri)
primary, secondary = service.get_keys()
print(primary)

http://8f8a4859-b934-4d82-b2f8-e49d3a8e5acc.southcentralus.azurecontainer.io/score
1Zr58HXlfjrfRk9S711H6LEwMqanCtvY


TODO: In the cell below, send a request to the web service you deployed to test it.

In [39]:
import urllib.request
import json
import os
import ssl

data = {
  "data":
        [
          {
           "age": 53, 
           "anaemia": 0, 
           "creatinine_phosphokinase": 332, 
           "diabetes": 0, 
           "ejection_fraction": 20, 
           "high_blood_pressure": 1, 
           "platelets": 11300, 
           "serum_creatinine": 1.9, 
           "serum_sodium": 130, 
           "sex": 1, 
           "smoking": 0,
           "time": 4
          },
          {
            "age": 22, 
           "anaemia": 1, 
           "creatinine_phosphokinase": 60, 
           "diabetes": 0, 
           "ejection_fraction": 50, 
           "high_blood_pressure": 0, 
           "platelets": 226000, 
           "serum_creatinine": 1, 
           "serum_sodium": 134, 
           "sex": 1, 
           "smoking": 0,
           "time": 30
          },
      ]
    }

body = str.encode(json.dumps(data))
#print(body)
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ primary)}

req = urllib.request.Request(service.scoring_uri, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(json.loads(result.decode("utf8", 'ignore')))
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

{"result": [1, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [33]:
service.get_logs()

'2021-02-08T06:59:34,722778740+00:00 - rsyslog/run \n2021-02-08T06:59:34,723572902+00:00 - iot-server/run \n2021-02-08T06:59:34,723752015+00:00 - gunicorn/run \n2021-02-08T06:59:34,730872064+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_75ee4ee71ca0e667d046665f3c52f2f8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_75ee4ee71ca0e667d046665f3c52f2f8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_75ee4ee71ca0e667d046665f3c52f2f8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_75ee4ee71ca0e667d046665f3c52f2f8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_75ee4ee71ca0e667d046665f3c52f2f8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [45]:
service.delete()

In [46]:
compute_target.delete()
compute_target.wait_for_completion(show_output=True)


Deleting